In [7]:
import os

# Install git if not already installed
!apt-get install -y git

# Clone the repository
!git clone https://github.com/Elite-AI-Club/AI-Driven-Innovation-Electronics.git

# Change working directory to the repository
%cd AI-Driven-Innovation-Electronics/7_PCB_Fault_Detection

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git is already the newest version (1:2.34.1-1ubuntu1.11).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.
Cloning into 'AI-Driven-Innovation-Electronics'...
remote: Enumerating objects: 1328, done.
remote: Counting objects: 100% (111/111), done.
remote: Compressing objects: 100% (75/75), done.
remote: Total 1328 (delta 32), reused 109 (delta 31), pack-reused 1217 (from 1)
Receiving objects: 100% (1328/1328), 109.40 MiB | 32.00 MiB/s, done.
Resolving deltas: 100% (36/36), done.
/content/AI-Driven-Innovation-Electronics/7_PCB_Fault_Detection


In [8]:
import os
import numpy as np
import pandas as pd
from PIL import Image
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
from flask import Flask, render_template, request
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array

In [9]:
app = Flask(__name__)

In [10]:
def get_model():
    global model
    model = load_model('model.h5')
    print("Model loaded!")

In [11]:
def load_image(img_path):

    img = image.load_img(img_path, target_size=(224, 224))
    img_tensor = image.img_to_array(img)                    # (height, width, channels)
    img_tensor = np.expand_dims(img_tensor, axis=0)         # (1, height, width, channels), add a dimension because the model expects this shape: (batch_size, height, width, channels)
    img_tensor /= 255.                                      # imshow expects values in the range [0, 1]

    return img_tensor

In [12]:
def prediction(img_path):
    new_image = load_image(img_path)

    pred = model.predict(new_image)

    print(pred)

    labels=np.array(pred)
    labels[labels>=0.6]=1
    labels[labels<0.6]=0

    print(labels)
    final=np.array(labels)

    if final[0][0]==1:
        return "Bad"
    else:
        return "Good"

In [13]:
get_model()

Model loaded!


In [14]:
@app.route("/", methods=['GET', 'POST'])
def home():

    return render_template('home.html')

@app.route("/predict", methods = ['GET','POST'])
def predict():

    if request.method == 'POST':

        file = request.files['file']
        filename = file.filename
        file_path = os.path.join(r'static/', filename)                       #slashes should be handeled properly
        file.save(file_path)
        print(filename)
        product = prediction(file_path)
        print(product)

    return render_template('predict.html', product = product, user_image = file_path)            #file_path can or may used at the place of filename

In [15]:
if __name__ == "__main__":
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
